<a href="https://colab.research.google.com/github/caseyhyoon/W266-Final-Project/blob/casey/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.model_selection import train_test_split
import string
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
os.listdir()

['.config',
 'twitter_sexism_parsed_dataset.csv',
 'twitter_racism_parsed_dataset.csv',
 'twitter_parsed_dataset.csv',
 'sample_data']

In [4]:
parsed = pd.read_csv('twitter_parsed_dataset.csv')
racism = pd.read_csv('twitter_racism_parsed_dataset.csv')
sexism = pd.read_csv('twitter_sexism_parsed_dataset.csv')

twitter_data = pd.concat([parsed, racism, sexism]).dropna()
twitter_data.head()

,index,id,Text,Annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0


In [5]:
### Cleaning tweets

def cleaning_tweets(tweet):
    # 1. Remove Twitter handles (@user)
    users = re.findall("@[\w]*", tweet) # tokenizing
    for user in users:
        tweet = re.sub(user, '', tweet)
        
    # 2. Remove urls
    tweet = re.sub(r'http\S+', '', tweet)

    # 3. Remove, Punctuations, Numbers, and Special Characters (keep hashtags)
    tweet = tweet.replace(".", " ").replace(",", " ").replace("?", " ").replace("!", " ")
    tweet = "".join([char for char in tweet if char not in string.punctuation])
    tweet = re.sub('[0-9]+', '', tweet)

    # 4. Lowercase all
    tweet = tweet.lower()
    
    return tweet

twitter_data['cleaned_tweets'] = twitter_data['Text'].apply(cleaning_tweets)
twitter_data.head()

,index,id,Text,Annotation,oh_label,cleaned_tweets
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0,i read them in context no change in meaning...
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0,now you idiots claim that people who tried to...
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0,rt call me sexist but when i go to an auto p...
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0,wrong isis follows the example of mohammed a...
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0,mkr no no no no no no


In [6]:
X = twitter_data['cleaned_tweets']
y = np.array(twitter_data['oh_label'])

sum(y)/len(y)

0.23660862446622563

In [7]:
X_train, X_test, y_train, y_test = train_test_split(twitter_data['cleaned_tweets'], twitter_data['oh_label'], test_size = 0.2, random_state=1)


Run for subset of data

In [6]:
twitter_subset = twitter_data.sample(10000)
x_subset = twitter_subset['cleaned_tweets']
y_subset = twitter_subset['oh_label']

X_train, X_test, y_train, y_test = train_test_split(x_subset, y_subset, test_size = 0.2, random_state=1)

In [ ]:
baseline = 1 - np.mean(y_train)
baseline

0.7541249999999999

In [8]:
pip install transformers

     |████████████████████████████████| 2.2MB 7.9MB/s 
     |████████████████████████████████| 870kB 52.6MB/s 
     |████████████████████████████████| 3.3MB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=77a7250b6856ba51f7925051ff1dad9475f5505191503ccfbd85c07d9243b255
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [9]:
from transformers import BertTokenizer, TFBertModel

In [10]:
bert_layer = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Freezing layers

In [11]:
retrain_layers = []

for retrain_layer_number in range(4):

    layer_code = '_' + str(11 - retrain_layer_number)
    retrain_layers.append(layer_code)

for w in bert_layer.weights:
    if not any([x in w.name for x in retrain_layers]):
        w._trainable = False

In [12]:
tf.keras.backend.clear_session()

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

bert_inputs = tokenizer(list(X_train), padding=True, return_tensors='tf')
dim = bert_inputs['input_ids'].shape[1]

In [14]:
input_ids = tf.keras.layers.Input(shape=(dim), dtype='int32', name='input_ids')
masks = tf.keras.layers.Input(shape=(dim), dtype='int32', name='mask')
token_type_ids = tf.keras.layers.Input(shape=(dim), dtype='int32', name='token_types')

bert_output = bert_layer([input_ids, masks, token_type_ids])


cls = bert_output[1]

hidden = tf.keras.layers.Dense(200, activation='relu')(cls)

classification = tf.keras.layers.Dense(1, activation='sigmoid')(hidden)

model = tf.keras.Model(inputs = [input_ids, masks, token_type_ids], outputs = classification)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(lr=0.0005, beta_1=0.91, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False),
              # optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics='acc')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [15]:
model.fit(x = [np.array(bert_inputs['input_ids']), np.array(bert_inputs['attention_mask']), np.array(bert_inputs['token_type_ids'])],
          y = y_train,
          epochs = 5,
          validation_split = 0.2,
          batch_size = 64)


Epoch 1/5
452/452 [==============================] - 118s 243ms/step - loss: 0.4087 - acc: 0.8206 - val_loss: 0.3085 - val_acc: 0.8704
Epoch 2/5
452/452 [==============================] - 108s 239ms/step - loss: 0.2980 - acc: 0.8733 - val_loss: 0.2874 - val_acc: 0.8819
Epoch 3/5
452/452 [==============================] - 108s 238ms/step - loss: 0.2683 - acc: 0.8872 - val_loss: 0.2759 - val_acc: 0.8879
Epoch 4/5
452/452 [==============================] - 108s 238ms/step - loss: 0.2501 - acc: 0.8948 - val_loss: 0.2675 - val_acc: 0.8931
Epoch 5/5
452/452 [==============================] - 108s 239ms/step - loss: 0.2373 - acc: 0.9007 - val_loss: 0.2659 - val_acc: 0.8952


In [16]:
bert_test_inputs = tokenizer(list(X_test), padding='max_length', truncation=True, max_length = dim, return_tensors='tf')
bert_test_inputs

{'input_ids': <tf.Tensor: shape=(9040, 49), dtype=int32, numpy=
array([[  101,   100,   102, ...,     0,     0,     0],
       [  101, 14145,  2080, ...,     0,     0,     0],
       [  101, 20228,  2480, ...,     0,     0,     0],
       ...,
       [  101,  2053,  2655, ...,     0,     0,     0],
       [  101,  2175, 29247, ...,     0,     0,     0],
       [  101,  2129,  2116, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(9040, 49), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(9040, 49), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32

In [17]:
y_pred = model.predict([np.array(bert_test_inputs['input_ids']), np.array(bert_test_inputs['attention_mask']), np.array(bert_test_inputs['token_type_ids'])])
y_pred = np.array([1 if x[0] >= 0.5 else 0  for x in y_pred])

print("accuracy: " + str(accuracy_score(y_test, y_pred)))

accuracy: 0.8934734513274336


In [18]:
errors = pd.DataFrame((y_pred == y_test))
error_index = np.array(errors[errors['oh_label'] == False].index)

for error in error_index[:15]:
  print(twitter_data.iloc[error]['Text'], twitter_data.iloc[error]['oh_label'])
  print()

i can't even @emk194 Call me sexist, but I actually prefer having guys in charge. Lol atmosphere is more laid-back &amp; fun. Women are serious 1.0

I need to just switch to an organization-based github, but I don't want to pay $25/month because I'm cheap. :\ 0.0

Things I am not scared of:

Whiny manbabies.
A lot of twitter mentions.
Public speaking.
Perl.
Standing up for those that are afraid.
FEELS. 0.0

@HMarston Get control of the northern border all the way to Efrin. Kobane and Efrin should be connected. 0.0

So she can try &amp; claim innocence, but statistics paint her as being someone to avoid. And I've been in her sights for a while. Oh well. 0.0

That's crap Lynn and Tony should have stayed #MKR 0.0

@qmarq used to be 0.0

RT @Linka_D: Kat is a mole #MKR 0.0

@desertcrossing And that proves what? 1.0

@Vandaliser @sajid_fairooz @IsraeliRegime Mohammed never looked for knowledge. He believed in Jinn, was illiterate, believed dogs ere devil. 1.0

RT @femfreq: Here is @PasteMag

In [19]:
twitter_data.iloc[error_index[0]]

index                                         3.84043241659589E+017
id                                            3.84043241659589E+017
Text              i can't even @emk194 Call me sexist, but I act...
Annotation                                                   sexism
oh_label                                                          1
cleaned_tweets    i cant even  call me sexist  but i actually pr...
Name: 13836, dtype: object